In [1]:
import numpy as np
from scipy.stats import uniform, norm, f, t
import pandas as pd
import statsmodels.api as sm
np.random.seed(314) 


In [7]:
n = 12
x_1 = uniform.rvs(loc = -1, scale = 2, size = n)
x_2 = uniform.rvs(loc = -3, scale = 4, size = n)
x_3 = uniform.rvs(loc = 1, scale = 9, size = n)
x_4 = uniform.rvs(loc = 1, scale = 9, size = n)
#x_5 = uniform.rvs(loc = -4, scale = 3, size = n)

model_error = norm.rvs(loc=0, scale = 1, size= n)
y = 8 + 4* x_1 -3*x_2 +0.5*x_3+3*x_4 + model_error

In [8]:
X = pd.DataFrame({"x_1":x_1,"x_2":x_2,"x_3":x_3,"x_4":x_4})
X = sm.add_constant(X)

df residual = no. observations - no. parameters (intercept is counted)

df model = no. regressors (intercept is not counted)

In [28]:
dof_res = len(X)-len(X.columns)
dof_model = len([col for col in X.columns if col!="const"])
print("dof_res: {}, dof_model: {}".format(dof_res, dof_model))

dof_res: 7, dof_model: 4


In [10]:
est = sm.OLS(y,X)
est_fit = est.fit()
print(est_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     223.5
Date:                Thu, 08 Jun 2023   Prob (F-statistic):           1.85e-07
Time:                        16:59:39   Log-Likelihood:                -12.256
No. Observations:                  12   AIC:                             34.51
Df Residuals:                       7   BIC:                             36.94
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.1365      1.166      7.837      0.0

c:\Users\pablo\Documents\clean_projects\statistics\myenv\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [25]:
lim_inf = t.ppf(0.025,est_fit.df_resid)*est_fit.bse+est_fit.params
lim_sup = t.ppf(0.975,est_fit.df_resid)*est_fit.bse+est_fit.params
tvalue= est_fit.params/est_fit.bse
pt = tvalue.apply(lambda x: 2 - 2*t.cdf(abs(x),est_fit.df_resid))
pd.DataFrame({"t":tvalue, "p>|t|":pt,"ci_inf":lim_inf,"ci_sum":lim_sup})

,t,p>|t|,ci_inf,ci_sum
const,7.836539,1.039674e-04,6.379641,11.893417
x_1,6.813254,2.501774e-04,2.666868,5.501962
x_2,-11.550692,8.210901e-06,-3.887145,-2.566063
x_3,2.557784,3.767323e-02,0.022247,0.566946
x_4,22.699612,8.154774e-08,2.630015,3.241670


In [24]:
{"model_sse":((est_fit.fittedvalues-y.mean())**2).sum(),
"residuals_sse":((est_fit.fittedvalues-y)**2).sum(),
"total_sse":((y.mean()-y)**2).sum()}

{'model_sse': 1703.3020971378014,
 'residuals_sse': 27.188903498525477,
 'total_sse': 1730.4910006363252}

In [6]:
#MSE
dfe = pd.DataFrame({"model":{"SME":est_fit.mse_model,"df":est_fit.df_model},
              "residuals":{"SME":est_fit.mse_resid,"df":est_fit.df_resid},
              "total":{"SME":est_fit.mse_total,"df":est_fit.df_model+est_fit.df_resid}
            }).T
dfe["SSE"]= dfe["SME"]*dfe["df"]

dfe

,SME,df,SSE
model,425.825524,4.0,1703.302097
residuals,1.087556,25.0,27.188903
total,59.672103,29.0,1730.491001


In [7]:
#R_squared
1-dfe.loc["residuals","SSE"]/dfe.loc["total","SSE"]

0.9842883300239482

In [8]:
#F-statistic
f_value = dfe.loc["model","SME"]/dfe.loc["residuals","SME"]

pf = 1 - f.cdf(f_value,dfe.loc["model","df"],dfe.loc["residuals","df"])

print("f={}, model_df = {}, residual_df = {} -> P>f = {}".format(f_value, dfe.loc["model","df"],dfe.loc["residuals","df"],pf))

f=391.5434878677101, model_df = 4.0, residual_df = 25.0 -> P>f = 1.1102230246251565e-16


In [9]:
est_fit.fvalue

391.5434878677101

In [98]:
est_fit.f_test

<bound method LikelihoodModelResults.f_test of <statsmodels.regression.linear_model.OLSResults object at 0x0000027E941CE260>>

In [13]:
est_fit.params

const    8.584818
x_1      4.148670
x_2     -3.043199
x_3      0.534070
x_4      2.901453
dtype: float64

27.188903498525477

In [12]:
est_fit.fittedvalues

0     39.225173
1     23.939333
2     32.075193
3     22.912338
4     28.848238
5     32.210296
6     35.483469
7     35.983511
8     33.136094
9     28.638358
10    47.994275
11    15.780199
12    27.981217
13    42.672202
14    16.750609
15    27.207327
16    21.915417
17    26.672734
18    27.603756
19    37.048320
20    28.021188
21    36.110934
22    24.662749
23    27.047579
24    25.131135
25    27.849770
26    46.611966
27    33.211652
28    32.965828
29    37.282262
dtype: float64